# Loading datasets from database (only user and post_text)

In [2]:
import pandas as pd
from dotenv import load_dotenv
import os


load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)


user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con=engine)

user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [4]:
post_text_df = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con=engine)

post_text_df

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


# Changing post_text - creating features

In [7]:
post_text_df['post_length'] = post_text_df['text'].apply(len)

In [9]:
post_text_df['num_hashtags'] = post_text_df['text'].apply(lambda x: x.count('#'))

In [11]:
post_text_df['num_mentions'] = post_text_df['text'].apply(lambda x: x.count('@'))

In [13]:
post_text_df['has_hashtag'] = post_text_df['text'].apply(lambda x: '#' in x)

In [15]:
post_text_df['has_url'] = post_text_df['text'].apply(lambda x: 'http' in x)

# Deep learning improvements

In [18]:
### 1. BERT-based Vectorization of Posts & Feature Extraction

In [20]:
from sentence_transformers import SentenceTransformer

In [22]:
### Loading BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [24]:
### Getting BERT embeddings

from tqdm import tqdm

# Apply tqdm to track encoding progress
tqdm.pandas(desc="Encoding Posts with BERT")

# Getting BERT embeddings with progress bar
post_text_df['post_embedding'] = post_text_df['text'].progress_apply(lambda x: bert_model.encode(x))

Encoding Posts with BERT: 100%|█████████████| 7023/7023 [04:39<00:00, 25.12it/s]


In [25]:
import numpy as np
# Converting embeddings into new features (mean, max, min)
post_text_df['embedding_mean'] = post_text_df['post_embedding'].apply(lambda x: np.mean(x))
post_text_df['embedding_max'] = post_text_df['post_embedding'].apply(lambda x: np.max(x))
post_text_df['embedding_min'] = post_text_df['post_embedding'].apply(lambda x: np.min(x))

In [28]:
# Drop raw embeddings (optional)
post_text_df.drop(columns=['post_embedding'], inplace=True)

In [30]:
post_text_df

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False,-0.000771,0.150771,-0.140247
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False,-0.001662,0.144552,-0.151125
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False,-0.001554,0.143036,-0.144040
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False,-0.000081,0.137932,-0.174149
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False,0.000010,0.162679,-0.154663
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False,-0.000816,0.164095,-0.128535
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False,-0.001117,0.138495,-0.138613
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False,-0.000882,0.152540,-0.209523
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False,-0.000055,0.187223,-0.138941


In [32]:
import numpy as np

post_text_df.loc[:,post_text_df.dtypes==np.object_].columns

Index(['text', 'topic'], dtype='object')

In [36]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


post_text_df.to_sql('daria_luzina_features_post_3', con=engine, index=False) # записываем таблицу

df = pd.read_sql('SELECT * FROM daria_luzina_features_post_3', con=engine)

In [38]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

df = pd.read_sql('SELECT * FROM daria_luzina_features_post_3', con=engine)

In [40]:
df

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False,-0.000771,0.150771,-0.140247
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False,-0.001662,0.144552,-0.151125
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False,-0.001554,0.143036,-0.144040
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False,-0.000081,0.137932,-0.174149
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False,0.000010,0.162679,-0.154663
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False,-0.000816,0.164095,-0.128535
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False,-0.001117,0.138495,-0.138613
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False,-0.000882,0.152540,-0.209523
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False,-0.000055,0.187223,-0.138941


In [42]:
import pandas as pd
from sqlalchemy import create_engine


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [44]:
def load_features() -> pd.DataFrame:
    query = "SELECT * FROM public.user_data"
    return batch_load_sql(query)


def load_features() -> pd.DataFrame:
    df_users = batch_load_sql("SELECT * from public.user_data")
    df_posts = batch_load_sql("SELECT * from daria_luzina_features_post_3")
    return df_users, df_posts

df_users, df_posts = load_features()

In [45]:
df_users

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [48]:
df_posts

,post_id,text,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1967,0,0,False,False,-0.000771,0.150771,-0.140247
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,2701,0,0,False,False,-0.001662,0.144552,-0.151125
2,3,Asian quake hits European shares\n\nShares in ...,business,3408,0,0,False,False,-0.001554,0.143036,-0.144040
3,4,India power shares jump on debut\n\nShares in ...,business,1026,0,0,False,False,-0.000081,0.137932,-0.174149
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,889,0,0,False,False,0.000010,0.162679,-0.154663
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,803,0,0,False,False,-0.000816,0.164095,-0.128535
7019,7316,I give this movie 2 stars purely because of it...,movie,800,0,0,False,False,-0.001117,0.138495,-0.138613
7020,7317,I cant believe this film was allowed to be mad...,movie,636,0,0,False,False,-0.000882,0.152540,-0.209523
7021,7318,The version I saw of this film was the Blockbu...,movie,728,0,0,False,False,-0.000055,0.187223,-0.138941


In [50]:
dataset = df_users[df_users['user_id']==200]

In [52]:
dataset = dataset.merge(df_posts.drop(columns='text'), how='cross')

In [54]:
dataset

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min
0,200,1,34,Russia,Degtyarsk,3,Android,ads,1,business,1967,0,0,False,False,-0.000771,0.150771,-0.140247
1,200,1,34,Russia,Degtyarsk,3,Android,ads,2,business,2701,0,0,False,False,-0.001662,0.144552,-0.151125
2,200,1,34,Russia,Degtyarsk,3,Android,ads,3,business,3408,0,0,False,False,-0.001554,0.143036,-0.144040
3,200,1,34,Russia,Degtyarsk,3,Android,ads,4,business,1026,0,0,False,False,-0.000081,0.137932,-0.174149
4,200,1,34,Russia,Degtyarsk,3,Android,ads,5,business,889,0,0,False,False,0.000010,0.162679,-0.154663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,200,1,34,Russia,Degtyarsk,3,Android,ads,7315,movie,803,0,0,False,False,-0.000816,0.164095,-0.128535
7019,200,1,34,Russia,Degtyarsk,3,Android,ads,7316,movie,800,0,0,False,False,-0.001117,0.138495,-0.138613
7020,200,1,34,Russia,Degtyarsk,3,Android,ads,7317,movie,636,0,0,False,False,-0.000882,0.152540,-0.209523
7021,200,1,34,Russia,Degtyarsk,3,Android,ads,7318,movie,728,0,0,False,False,-0.000055,0.187223,-0.138941


In [56]:
dataset = dataset.drop(columns = 'user_id')

In [58]:
dataset = dataset.set_index('post_id')

In [60]:
dataset

,gender,age,country,city,exp_group,os,source,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min
post_id,,,,,,,,,,,,,,,,
1,1,34,Russia,Degtyarsk,3,Android,ads,business,1967,0,0,False,False,-0.000771,0.150771,-0.140247
2,1,34,Russia,Degtyarsk,3,Android,ads,business,2701,0,0,False,False,-0.001662,0.144552,-0.151125
3,1,34,Russia,Degtyarsk,3,Android,ads,business,3408,0,0,False,False,-0.001554,0.143036,-0.144040
4,1,34,Russia,Degtyarsk,3,Android,ads,business,1026,0,0,False,False,-0.000081,0.137932,-0.174149
5,1,34,Russia,Degtyarsk,3,Android,ads,business,889,0,0,False,False,0.000010,0.162679,-0.154663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,1,34,Russia,Degtyarsk,3,Android,ads,movie,803,0,0,False,False,-0.000816,0.164095,-0.128535
7316,1,34,Russia,Degtyarsk,3,Android,ads,movie,800,0,0,False,False,-0.001117,0.138495,-0.138613
7317,1,34,Russia,Degtyarsk,3,Android,ads,movie,636,0,0,False,False,-0.000882,0.152540,-0.209523


In [62]:
import os
from catboost import CatBoostClassifier

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("/Users/daraluzina/ML/HW_22/Ver_2.0/catboost_model_4.cbm")  
    
    # Load CatBoost model
    model = CatBoostClassifier()
    model.load_model(model_path)
    
    return model

In [64]:
model_test = load_models()

In [66]:
model_test.predict_proba(dataset)[:,1]

array([0.09608453, 0.10579442, 0.10236344, ..., 0.13947349, 0.16000311,
       0.13295646])

In [68]:
dataset['predict_proba'] = model_test.predict_proba(dataset)[:,1]

In [70]:
dataset = dataset.sort_values(by='predict_proba', ascending=False).head(5)

In [72]:
dataset

,gender,age,country,city,exp_group,os,source,topic,post_length,num_hashtags,num_mentions,has_hashtag,has_url,embedding_mean,embedding_max,embedding_min,predict_proba
post_id,,,,,,,,,,,,,,,,,
6542,1,34,Russia,Degtyarsk,3,Android,ads,movie,1810,0,0,False,False,0.000334,0.196419,-0.128496,0.225866
5263,1,34,Russia,Degtyarsk,3,Android,ads,movie,988,0,0,False,False,0.000676,0.184041,-0.113304,0.224685
7106,1,34,Russia,Degtyarsk,3,Android,ads,movie,1888,0,0,False,False,-0.001243,0.200487,-0.169293,0.218615
5764,1,34,Russia,Degtyarsk,3,Android,ads,movie,639,0,0,False,False,0.000425,0.175364,-0.124365,0.217779
7054,1,34,Russia,Degtyarsk,3,Android,ads,movie,1889,0,0,False,False,0.000201,0.185431,-0.127248,0.215950


In [74]:
dataset = dataset.merge(df_posts, left_index=True, right_on = 'post_id').reset_index()

In [76]:
dataset

,index,gender,age,country,city,exp_group,os,source,topic_x,post_length_x,...,text,topic_y,post_length_y,num_hashtags_y,num_mentions_y,has_hashtag_y,has_url_y,embedding_mean_y,embedding_max_y,embedding_min_y
0,6284,1,34,Russia,Degtyarsk,3,Android,ads,movie,1810,...,I know that there are some purists out there w...,movie,1810,0,0,False,False,0.000334,0.196419,-0.128496
1,5069,1,34,Russia,Degtyarsk,3,Android,ads,movie,988,...,"Hell to Pay is a bit bloody for my taste, even...",movie,988,0,0,False,False,0.000676,0.184041,-0.113304
2,6825,1,34,Russia,Degtyarsk,3,Android,ads,movie,1888,...,Margret Laurence probably didnt intend on havi...,movie,1888,0,0,False,False,-0.001243,0.200487,-0.169293
3,5547,1,34,Russia,Degtyarsk,3,Android,ads,movie,639,...,I thought this series was going to be another ...,movie,639,0,0,False,False,0.000425,0.175364,-0.124365
4,6779,1,34,Russia,Degtyarsk,3,Android,ads,movie,1889,...,The problem with so many people watching this ...,movie,1889,0,0,False,False,0.000201,0.185431,-0.127248


In [78]:
dataset = dataset[['post_id','text','topic_x']]


In [80]:
dataset

,post_id,text,topic_x
0,6542,I know that there are some purists out there w...,movie
1,5263,"Hell to Pay is a bit bloody for my taste, even...",movie
2,7106,Margret Laurence probably didnt intend on havi...,movie
3,5764,I thought this series was going to be another ...,movie
4,7054,The problem with so many people watching this ...,movie


In [82]:
list_of_dicts = dataset.to_dict('records')
list_of_dicts

[{'post_id': 6542,
  'text': 'I know that there are some purists out there who poo poo anything that is not exactly like the original, however sometimes spin-offs can stand on their own merits. I like the new Iron Chef because it is similar enough to the Japanese version but at the same time caters to American spirit. I love Alton Brown as commentator, because he explains things with flair. The Iron Chefs themselves are very interesting. I know the originals were probably the best chefs on the planet at the time, but Bobby Flay is the only American Iron Chef to beat them. Mario Batali seems to have the most fun when cooking, making comments and being flashy while creating. I have watched the series and find all the players work together well. The judges are not always the best choices, however. There are a few exceptions, like the lawyer turned foodie, but most of the judges are questionable in being able to handle what is served. I enjoy watching the chefs hustle and the challengers a